In [61]:
import os
import imghdr
import itertools
from shutil import copyfile

cwd = os.getcwd()

frames = ["HazzBerry", "GerretLight", "Enzo"]
colors = ['obsidianBlack', 'glamourRed', 'goldPotato', 'tornadoBlue', 'lushGreen']

result_dir_name = 'processed_data'

if not os.path.exists(result_dir_name):
        os.makedirs(result_dir_name)

# Make folders for each
folder_names = []

for pair in itertools.product(frames, colors):
    folder_name = '_'.join(pair)
    folder_names.append(folder_name)
    print(folder_name)
    directry_name = result_dir_name +'/' + folder_name
    if not os.path.exists(directry_name):
        os.makedirs(directry_name)
    
subfolders = os.listdir(os.path.join(cwd,'batch'))

print(subfolders)

for sub in subfolders:
    print(sub)
    if(sub != '.DS_Store'):
        print(sub)
        print()
        for subdir, dirs, files in os.walk(cwd+'/batch/' + sub):
            print(len(files))
            #for sd in dirs:
            for file in files:
                if("mask" in file):
                    continue
                if(imghdr.what(os.path.join(subdir, file)) == 'png'):
                    print(file)
                    if('_' in file):
                        file_elements = file.split("_")
                        folder = file_elements[1] + '_' + file_elements[2]
                        print(folder)
                        copyfile(cwd+'/batch/' + sub + '/' + file, cwd + '/' + result_dir_name + '/' + folder + '/' +  sub + '_' + file)
                        
                   # CHECK IF NAME IS 

                #print(os.path.join(subdir, file))


# For each folder
# Sort through all images
# Get a list of references of all that have EN

HazzBerry_obsidianBlack
HazzBerry_glamourRed
HazzBerry_goldPotato
HazzBerry_tornadoBlue
HazzBerry_lushGreen
GerretLight_obsidianBlack
GerretLight_glamourRed
GerretLight_goldPotato
GerretLight_tornadoBlue
GerretLight_lushGreen
Enzo_obsidianBlack
Enzo_glamourRed
Enzo_goldPotato
Enzo_tornadoBlue
Enzo_lushGreen
['.DS_Store', '0', '1', '2']
.DS_Store
0
0

170
default_Enzo_lushGreen_p0_y0_r0.png
Enzo_lushGreen
default_Enzo_goldPotato_p0_y0_r0.png
Enzo_goldPotato
default_HazzBerry_obsidianBlack_p5_y0_r0.png
HazzBerry_obsidianBlack
default_GerretLight_tornadoBlue_p0_y0_r0.png
GerretLight_tornadoBlue
default_HazzBerry_tornadoBlue_p5_y0_r0.png
HazzBerry_tornadoBlue
default_HazzBerry_glamourRed_p5_y10_r0.png
HazzBerry_glamourRed
default_HazzBerry_obsidianBlack_p0_y10_r0.png
HazzBerry_obsidianBlack
default_GerretLight_goldPotato_p-5_y0_r0.png
GerretLight_goldPotato
default_HazzBerry_obsidianBlack_p-5_y0_r0.png
HazzBerry_obsidianBlack
default_GerretLight_goldPotato_p0_y10_r0.png
GerretLight_goldPot

default_GerretLight_tornadoBlue_p-5_y-10_r0.png
GerretLight_tornadoBlue
default_HazzBerry_goldPotato_p0_y0_r0.png
HazzBerry_goldPotato
default_GerretLight_glamourRed_p5_y0_r0.png
GerretLight_glamourRed
default_GerretLight_glamourRed_p-5_y10_r0.png
GerretLight_glamourRed
default_Enzo_glamourRed_p-5_y0_r0.png
Enzo_glamourRed
default_HazzBerry_obsidianBlack_p0_y-10_r0.png
HazzBerry_obsidianBlack
default_GerretLight_glamourRed_p5_y-10_r0.png
GerretLight_glamourRed
default_HazzBerry_tornadoBlue_p-5_y-10_r0.png
HazzBerry_tornadoBlue
default_HazzBerry_tornadoBlue_p0_y10_r0.png
HazzBerry_tornadoBlue
default_Enzo_glamourRed_p5_y10_r0.png
Enzo_glamourRed
default_Enzo_glamourRed_p0_y-10_r0.png
Enzo_glamourRed
default_HazzBerry_tornadoBlue_p-5_y0_r0.png
HazzBerry_tornadoBlue
default_HazzBerry_obsidianBlack_p-5_y10_r0.png
HazzBerry_obsidianBlack
default_Enzo_glamourRed_p-5_y10_r0.png
Enzo_glamourRed
default_HazzBerry_lushGreen_p5_y-10_r0.png
HazzBerry_lushGreen
default_Enzo_goldPotato_p5_y-10_r0.pn

default_HazzBerry_lushGreen_p0_y-10_r0.png
HazzBerry_lushGreen
default_GerretLight_tornadoBlue_p-5_y-10_r0.png
GerretLight_tornadoBlue
default_HazzBerry_goldPotato_p0_y0_r0.png
HazzBerry_goldPotato
default_GerretLight_glamourRed_p5_y0_r0.png
GerretLight_glamourRed
default_GerretLight_glamourRed_p-5_y10_r0.png
GerretLight_glamourRed
default_Enzo_glamourRed_p-5_y0_r0.png
Enzo_glamourRed
default_HazzBerry_obsidianBlack_p0_y-10_r0.png
HazzBerry_obsidianBlack
default_GerretLight_glamourRed_p5_y-10_r0.png
GerretLight_glamourRed
default_HazzBerry_tornadoBlue_p-5_y-10_r0.png
HazzBerry_tornadoBlue
default_HazzBerry_tornadoBlue_p0_y10_r0.png
HazzBerry_tornadoBlue
default_Enzo_glamourRed_p5_y10_r0.png
Enzo_glamourRed
default_Enzo_glamourRed_p0_y-10_r0.png
Enzo_glamourRed
default_HazzBerry_tornadoBlue_p-5_y0_r0.png
HazzBerry_tornadoBlue
default_HazzBerry_obsidianBlack_p-5_y10_r0.png
HazzBerry_obsidianBlack
default_Enzo_glamourRed_p-5_y10_r0.png
Enzo_glamourRed
default_HazzBerry_lushGreen_p5_y-10_r